In [5]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import uuid
import csv
from datetime import timedelta
import datetime
import time

flight_memo = {
    'flight_uuid' : ''
    ,'memo' : ''
}

flight_price = {
    'flight_uuid' : ''
    ,'single_price' : ''
}

go_plan = {
    'flight_uuid' : ''
    ,'company' : ''
    ,'trans_plan':[]
    ,'trans_cnt' : ''
    ,'takeoff_time' : ''
    ,'land_time' :''
    ,'takeoff_location' : ''
    ,'land_location' : ''
    ,'spend_time' : ''
    ,'spend_hr' : ''
    ,'spend_min' : ''
}

bk_plan = {
    'flight_uuid' : ''
    ,'company' : ''
    ,'trans_plan': []
    ,'trans_cnt' : ''
    ,'takeoff_time' : ''
    ,'land_time': ''
    ,'takeoff_location' : ''
    ,'land_location' : ''
    ,'spend_time' : ''
    ,'spend_hr' : ''
    ,'spend_min' : ''
}

options = uc.ChromeOptions()
#options.headless=True  
#options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = uc.Chrome(options=options)

takeoff_location = 'tpet'
land_location = 'hkd'
go_plan_date = '230805'
bk_plan_date = '230831'
people = '2'

browser.get(f'https://www.skyscanner.com.tw/transport/flights/{takeoff_location}/{land_location}/{go_plan_date}/{bk_plan_date}/?adultsv2={people}')
time.sleep(35)
# 點擊顯示更多結果
browser.find_element(By.XPATH, '//*[@id="app-root"]/div[1]/div/div/div/div[1]/button').click()
past_height = ''
current_height = ''
while True:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
    current_height = browser.execute_script("return document.body.scrollHeight")
    print('current_height:',current_height)
    print('past_height:',past_height)
    if past_height == current_height:
        print('Closed for now')
        break
    else:
        past_height = current_height
        print('網頁更新中...')

with open('./flight_memo.csv','w',newline='', encoding="utf8") as flight_memo_file,\
     open('./flight_price.csv','w',newline='', encoding="utf8") as flight_price_file,\
     open('./go_plan.csv','w',newline='', encoding="utf8") as go_plan_file,\
     open('./bk_plan.csv','w',newline='', encoding="utf8") as bk_plan_file:
    flight_memo_writer = csv.DictWriter(flight_memo_file, fieldnames=[*flight_memo.keys()], delimiter="|")
    flight_price_writer = csv.DictWriter(flight_price_file, fieldnames=[*flight_price.keys()], delimiter="|")
    go_plan_writer = csv.DictWriter(go_plan_file, fieldnames=[*go_plan.keys()], delimiter="|")
    bk_plan_writer = csv.DictWriter(bk_plan_file, fieldnames=[*bk_plan.keys()], delimiter="|")
    
    flight_memo_writer.writeheader()
    flight_price_writer.writeheader()
    go_plan_writer.writeheader()
    bk_plan_writer.writeheader()
    
    html_source = browser.page_source
    browser.quit()
    soup = BeautifulSoup(html_source, "html.parser")
    memo = soup.find_all("div",class_ = 'UpperTicketBody_screenReaderOnly__YTY2Z') #敘述整體航班資訊
    uuid_list = []
    for i in memo:
        flight_uuid = uuid.uuid4() #建立航班的PK用
        uuid_list.append(flight_uuid)
        # 處理memo資訊
        flight_memo['flight_uuid'] = flight_uuid
        flight_memo['memo'] = i.text.split('：')[1]
        flight_memo_writer.writerow(flight_memo)
        
        # 處理航班價格資訊
        flight_price['flight_uuid'] = flight_uuid
        flight_price['single_price'] = ''.join(filter(str.isdigit, i.text.split('：')[1].split('。')[0].split('；')[0])) #單人費用
        flight_price_writer.writerow(flight_price)
        print('memo:',flight_memo)
        print('flight_price:',flight_price)
        print('------------------------------------------------------------------------')
    products = soup.find_all("div",class_ = 'UpperTicketBody_legsContainer__ZjcyZ') #航班去回程時間/轉機等資訊
    cnt = 0
    for k in products:
        uuid_pk = uuid_list[cnt]
        go_plan['flight_uuid'] = uuid_pk
        print('開始處理去程資訊:')
        go_info = k.find_all("div",class_ = 'LegDetails_container__MTkyZ UpperTicketBody_leg__MmNkN')[0] #去程
        #去程航空公司(如果有2班以上的航空公司,以文字呈現,一間以圖片呈現,抓alt文字)
        company = go_info.find("div",class_ = 'LogoImage_container__MDU0Z LegLogo_logoContainer__ODdkM UpperTicketBody_legLogo__ZjYwM')
        try:
            go_plan['company'] = company.find("div",class_ = 'BpkImage_bpk-image__YTkyO BpkImage_bpk-image--no-background__NGMyN').img['alt']
        except:
            go_plan['company'] = company.text
        
        #去程轉機地點
        trans_location = go_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopStation__M2E5N')
        for location in trans_location:
            go_plan['trans_plan'].append(location.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY').text)
        if not go_plan['trans_plan']:
            no_trans = go_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopsLabelGreen__YWM4M')
            go_plan['trans_plan'].append(no_trans.text) #直飛
        
        #去程轉機次數    
        try:
            trans_cnt = go_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopsLabelRed__NTY2Y')
            go_plan['trans_cnt'] = re.search(r'\d+', trans_cnt.text).group()
        except:
            go_plan['trans_cnt'] = 0

        #去程花費時間    
        go_spend = go_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY Duration_duration__NmUyM').text
        go_plan['spend_time'] = go_spend
        try:
            spend_hr = int(''.join(filter(str.isdigit, go_spend.split('小時')[0])))
            go_plan['spend_hr'] = spend_hr
        except:
            go_plan['spend_hr'] = 0
        try:
            spend_min = int(''.join(filter(str.isdigit, go_spend.split('小時')[1])))
            go_plan['spend_min'] = spend_min
        except:
            go_plan['spend_min'] = 0
        #去程出發、抵達時間 
        takeoff_time = go_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--subheading__NzkwO')[0].text
        takeoff_datetime_tmp = go_plan_date + ' ' + takeoff_time
        takeoff_datetime = datetime.datetime.strptime(takeoff_datetime_tmp, '%y%m%d %H:%M')
        land_datetime = takeoff_datetime + timedelta(hours = spend_hr) + timedelta(minutes = spend_min)
        go_plan['takeoff_time'] = takeoff_datetime
        go_plan['land_time'] = land_datetime
        
        #去程的出發地&目的地
        takeoff_location = go_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--body-default__Y2M3Z LegInfo_routePartialCityTooltip__NTE4Z')[0]
        land_location = go_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--body-default__Y2M3Z LegInfo_routePartialCityTooltip__NTE4Z')[1]
        go_plan['takeoff_location'] = takeoff_location.text
        go_plan['land_location'] = land_location.text
        go_plan_writer.writerow(go_plan)
        print('go_plan:',go_plan)
        go_plan['trans_plan'].clear()
        print('---------------------------------------------------------------------------------------------')
        print('回程資訊:')
        bk_plan['flight_uuid'] = uuid_pk
        back_info = k.find_all("div",class_ = 'LegDetails_container__MTkyZ UpperTicketBody_leg__MmNkN')[1] #回程
        #回程航空公司(如果有2班以上的航空公司,以文字呈現,一間以圖片呈現,抓alt文字)
        company = back_info.find("div",class_ = 'LogoImage_container__MDU0Z LegLogo_logoContainer__ODdkM UpperTicketBody_legLogo__ZjYwM')
        try:
            bk_plan['company'] = company.find("div",class_ = 'BpkImage_bpk-image__YTkyO BpkImage_bpk-image--no-background__NGMyN').img['alt']
        except:
            bk_plan['company'] = company.text
        
        #回程轉機地點
        trans_location = back_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopStation__M2E5N')
        for location in trans_location:
            bk_plan['trans_plan'].append(location.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY').text)
        if not bk_plan['trans_plan']:
            no_trans = back_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopsLabelGreen__YWM4M')
            bk_plan['trans_plan'].append(no_trans.text) #直飛
        
        #回程轉機次數
        try:
            trans_cnt = back_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY LegInfo_stopsLabelRed__NTY2Y')
            bk_plan['trans_cnt'] = re.search(r'\d+', trans_cnt.text).group()
        except:
            bk_plan['trans_cnt'] = 0
        
        #回程花費時間    
        go_spend = back_info.find("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--xs__ZDJmY Duration_duration__NmUyM').text
        bk_plan['spend_time'] = go_spend
        try:
            spend_hr = int(''.join(filter(str.isdigit, go_spend.split('小時')[0])))
            bk_plan['spend_hr'] = spend_hr
        except:
            bk_plan['spend_hr'] = 0
        try:
            spend_min = int(''.join(filter(str.isdigit, go_spend.split('小時')[1])))
            bk_plan['spend_min'] = spend_min
        except:
            bk_plan['spend_min'] = 0
        
        #去程出發、抵達時間 
        takeoff_time = back_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--subheading__NzkwO')[0].text
        takeoff_datetime_tmp = bk_plan_date + ' ' + takeoff_time
        takeoff_datetime = datetime.datetime.strptime(takeoff_datetime_tmp, '%y%m%d %H:%M')
        land_datetime = takeoff_datetime + timedelta(hours = spend_hr) + timedelta(minutes = spend_min)
        bk_plan['takeoff_time'] = takeoff_datetime
        bk_plan['land_time'] = land_datetime
        
        #回程的出發地&目的地
        takeoff_location = back_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--body-default__Y2M3Z LegInfo_routePartialCityTooltip__NTE4Z')[0]
        land_location = back_info.find_all("span",class_ = 'BpkText_bpk-text__MWZkY BpkText_bpk-text--body-default__Y2M3Z LegInfo_routePartialCityTooltip__NTE4Z')[1]
        bk_plan['takeoff_location'] = takeoff_location.text
        bk_plan['land_location'] = land_location.text
        bk_plan_writer.writerow(bk_plan)
        print('bk_plan:',bk_plan)
        bk_plan['trans_plan'].clear()
        cnt+=1
    
        print('---------------------------------------------------------------------------------------------')
        print('---------------------------------------------------------------------------------------------')

current_height: 8105
past_height: 
網頁更新中...
current_height: 10659
past_height: 8105
網頁更新中...
current_height: 13215
past_height: 10659
網頁更新中...
current_height: 15729
past_height: 13215
網頁更新中...
current_height: 18325
past_height: 15729
網頁更新中...
current_height: 20951
past_height: 18325
網頁更新中...
current_height: 23461
past_height: 20951
網頁更新中...
current_height: 26099
past_height: 23461
網頁更新中...
current_height: 28551
past_height: 26099
網頁更新中...
current_height: 31025
past_height: 28551
網頁更新中...
current_height: 33393
past_height: 31025
網頁更新中...
current_height: 36085
past_height: 33393
網頁更新中...
current_height: 38559
past_height: 36085
網頁更新中...
current_height: 40971
past_height: 38559
網頁更新中...
current_height: 43355
past_height: 40971
網頁更新中...
current_height: 45881
past_height: 43355
網頁更新中...
current_height: 48197
past_height: 45881
網頁更新中...
current_height: 50573
past_height: 48197
網頁更新中...
current_height: 52909
past_height: 50573
網頁更新中...
current_height: 55283
past_height: 52909
網頁更新中...
current

In [ ]:
# 全球機場代碼
# http://www.ting.com.tw/tour-info/air-name.htm